## News Feed Extraction & Classification

In [4]:
import pandas as pd
import sys
sys.path.append('..')
from pydantic import BaseModel, Field
from typing import Literal, Optional
from utils.prompts import render
from utils.llm_client import LLMClient
from utils.router import pick_model
from IPython.display import Markdown, display

In [5]:
class CrisisEvent(BaseModel):
    district: Literal["Colombo", "Gampaha", "Kandy", "Kalutara", "Galle", "Matara", "Hambantota", "Jaffna", "Anuradhapura", "Polonnaruwa", "Badulla", "Monaragala", "Ratnapura", "Kegalle", "Nuwara Eliya", "Trincomalee", "Batticaloa", "Ampara", "Puttalam", "Kurunegala", "Mannar", "Mullaitivu", "Kilinochchi", "Vavuniya"]
    flood_level_meters: Optional[float] = None
    vicm_count: int = Field(default=0)
    main_need: str
    status: Literal["Critical", "Warning", "Stable"]

In [6]:
# Load news feed
file_path = "../data/news_feed.txt"

with open(file_path, "r", encoding="utf-8") as f:
    news_items = [line.strip() for line in f if line.strip()]

print(f"Loaded {len(news_items)} news items")
news_items[:3]

Loaded 30 news items


['BREAKING: Water levels in Kelani River (Colombo) have reached 9.5 meters. Critical flood warning issued.',
 'SOS: 5 people trapped on a roof in Ja-Ela (Gampaha). Water rising fast. Need boat immediately.',
 'Update: Kandy road cleared near Peradeniya. Traffic moving slowly. No victims reported.']

In [7]:
# Initialize LLM client
model = pick_model('openai', 'general')
client = LLMClient('openai', model)

# Schema for extraction
schema_str = """
{
  "district": "string (one of: Colombo, Gampaha, Kandy, Kalutara, Galle, Matara, Hambantota, Jaffna, Anuradhapura, Polonnaruwa, Badulla, Monaragala, Ratnapura, Kegalle, Nuwara Eliya, Trincomalee, Batticaloa, Ampara, Puttalam, Kurunegala, Mannar, Mullaitivu, Kilinochchi, Vavuniya)",
  "flood_level_meters": "number or null",
  "vicm_count": "integer (default 0)",
  "main_need": "string",
  "status": "string (one of: Critical, Warning, Stable)"
}
"""

In [14]:
# Process each news item
valid_events = []

for idx, item in enumerate(news_items, start=1):
    try:
        # Extract JSON using LLM
        prompt_text, spec = render(
            'json_extract.v1',
            schema=schema_str,
            text=item
        )
        
        messages = [{'role': 'user', 'content': prompt_text}]
        response = client.chat(messages, temperature=spec.temperature, max_tokens=spec.max_tokens)
        
        json_str = response["text"].strip()
        
        # Validate with Pydantic
        event = CrisisEvent.model_validate_json(json_str)
        valid_events.append(event.model_dump())
        
        print(f"Item {idx}: Valid - {event.district}")
        
    except Exception as e:
        print(f"Item {idx}: Invalid - {str(e)}")

# Convert to DataFrame
df = pd.DataFrame(valid_events)

# Save to Excel
output_path = "../outputs/flood_report.xlsx"
df.to_excel(output_path, index=False)

print(f"Saved {len(valid_events)} valid events to {output_path}")

Item 1: Valid - Colombo
Item 2: Valid - Gampaha
Item 3: Invalid - 1 validation error for CrisisEvent
  Invalid JSON: expected value at line 1 column 1 [type=json_invalid, input_value='```json\n{\n  "district"...atus": "Stable"\n}\n```', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/json_invalid...
Item 4: Invalid - 1 validation error for CrisisEvent
  Invalid JSON: expected value at line 1 column 1 [type=json_invalid, input_value='```json\n{\n  "district"...us": "Critical"\n}\n```', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/json_invalid...
Item 5: Valid - Gampaha
Item 6: Invalid - 1 validation error for CrisisEvent
  Invalid JSON: expected value at line 1 column 1 [type=json_invalid, input_value='```json\n{\n  "district"...atus": "Stable"\n}\n```', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/json_invalid...
Item 7: Valid - Matara
Item 8: Valid - Colombo
Item 9: Inv

KeyboardInterrupt: 